打包文件

In [5]:
import os
import zipfile
 
zipname='hhhsqlite.zip'

def zipDir(dirpath, outFullName):
    """
    压缩指定文件夹
    :param dirpath: 目标文件夹路径
    :param outFullName: 压缩文件保存路径+xxxx.zip
    :return: 无
    """
    zip = zipfile.ZipFile(outFullName, "w", zipfile.ZIP_DEFLATED)
    for path, dirnames, filenames in os.walk(dirpath):
        # 去掉目标跟路径，只对目标文件夹下边的文件及文件夹进行压缩
        fpath = path.replace(dirpath, '')
 
        for filename in filenames:
            zip.write(os.path.join(path, filename), os.path.join(fpath, filename))
    zip.close()
 
 
if __name__ == "__main__":
    input_path = "./sqlite"
    output_path =zipname
 
    zipDir(input_path, output_path)

发送到指定邮箱

In [10]:
import yagmail
import datetime

emailhost = "smtp.qq.com"
emailname = "473161189@qq.com"
emailpassword = "hspxxpupkwowbggg"  # 授权码 需要到邮箱的设置里面 开启授权

# 创建客户端
yag = yagmail.SMTP(
    user=emailname,
    password=emailpassword,
    host=emailhost,  # 邮局的 smtp 地址
    smtp_ssl=True # 启用 SSL 加密
)
# 发送邮件
yag.send(to=emailname,
         subject='为什么要演奏春日影？',
         contents='哈喽，你好！\n\n这是你的爬虫sqlite文件,现在是 '+f"{str(datetime.datetime.now())}",
         attachments=['preprocess.ipynb'])
# 关闭 yagmail 客户端
yag.close()

In [7]:
# 删除文件
import os

os.remove(zipname)

final py

In [ ]:
import os
import zipfile
import yagmail
import datetime
import time

namefile = 'Tempzip'

def zipDir(dirpath, outFullName, zip_limit_size):
    """
    压缩指定文件夹并限制每个压缩包的大小
    :param dirpath: 目标文件夹路径
    :param outFullName: 压缩文件保存路径+xxxx.zip
    :param zip_limit_size: 单个压缩包限制大小（字节）
    """
    zip_index = 1
    current_zip_size = 0
    zip_file = zipfile.ZipFile(f"{outFullName}_{zip_index}.zip", "w", zipfile.ZIP_DEFLATED)

    for path, dirnames, filenames in os.walk(dirpath):
        for filename in filenames:
            file_path = os.path.join(path, filename)
            file_size = os.path.getsize(file_path)

            # 检查当前压缩包是否超出限制
            if current_zip_size + file_size > zip_limit_size:
                zip_file.close()  # 关闭当前压缩包
                zip_index += 1   # 更新压缩包索引
                current_zip_size = 0  # 重置当前大小
                zip_file = zipfile.ZipFile(f"{outFullName}_{zip_index}.zip", "w", zipfile.ZIP_DEFLATED)  # 新建压缩包

            zip_file.write(file_path, os.path.relpath(file_path, dirpath))  # 添加文件
            current_zip_size += file_size  # 更新当前压缩大小

    zip_file.close()  # 压缩完成，关闭压缩包


if __name__ == "__main__":
    # 清除Tempzip文件夹里的所有文件
    for root, dirs, files in os.walk(namefile, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))
    namefile = 'Tempzip'
    emailhost = "smtp.qq.com"
    emailname = "473161189@qq.com"
    emailpassword = "hspxxpupkwowbggg"  # 授权码 需要到邮箱的设置里面 开启授权

    # 新建一个打包的文件夹，如果已存在则删除新加
    if os.path.exists(namefile):
        os.rmdir(namefile)

    os.mkdir(namefile)

    input_path = "./sqlite"
    output_path = namefile + '/hhhsqlite'
    zip_limit_size = 100 * 1024 * 1024  # 假设每个压缩包限制为30MB

    # 压缩文件夹
    zipDir(input_path, output_path, zip_limit_size)

    # 创建客户端
    yag = yagmail.SMTP(
        user=emailname,
        password=emailpassword,
        host=emailhost,
        smtp_ssl=True
    )

    # 发送所有生成的压缩包
    zip_index = 1
    while True:
        zip_file_name = f"{output_path}_{zip_index}.zip"
        if not os.path.exists(zip_file_name):  # 如果没有更多文件可发送
            break

        # 发送邮件
        yag.send(
            to=emailname,
            subject='为什么要演奏春日影？ '+str(zip_index),
            contents=f'哈喽，你好！\n\n这是你的爬虫sqlite文件,现在是 {datetime.datetime.now()}',
            attachments=[zip_file_name]
        )
        time.sleep(20)  # 延迟1秒，避免发送过快被服务器拒绝
        zip_index += 1  # 发送下一个压缩包

    # 关闭 yagmail 客户端
    yag.close()

    # 清除Tempzip文件夹里的所有文件
    for root, dirs, files in os.walk(namefile, topdown=False):
        for name in files:
            os.remove(os.path.join(root, name))
        for name in dirs:
            os.rmdir(os.path.join(root, name))


解压收到的压缩包文件到指定的目录

In [ ]:
# 解压一个文件夹里的所有文件至一个指定目录
import os
import shutil

# 压缩文件所在目录
src_dir = r'D:\Code\Python\PaChong\Crawlpreprocess\Tempzip'
# 解压文件存放目录
dst_dir = r'D:\Code\Python\PaChong\Crawlpreprocess\sqlite'

# 遍历src_dir目录下所有文件
for file in os.listdir(src_dir):
    # 判断是否为文件
    if os.path.isfile(os.path.join(src_dir, file)):
        # 解压文件至dst_dir目录
        shutil.unpack_archive(os.path.join(src_dir, file), dst_dir)